# Graph Neural Networks

### Author: [John F. Wu](https://jwuphysics.github.io/)

Presented as a tutorial during [Session 19](https://github.com/LSSTC-DSFP/Session-19) of the [LSSTC Data Science Fellowship Program](https://www.lsstcorporation.org/lincc/fellowship_program).

[![Open this notebook in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jwuphysics/LSSTC-DSFP-Session-19/blob/main/day4/Graph%20Neural%20Networks.ipynb)


## Introduction

In this tutorial, we will:
0. learn about the basics of mathematic graphs and their ability to represent (astro)physical systems,
1. create a cosmic graph from a catalog of dark matter halo properties,
2. link together neighboring halos on the graph, and
3. optimize a neural network that takes the cosmic graph as input, and outputs the stellar mass for every dark matter halo.


<center><img src="https://ar5iv.labs.arxiv.org/html/2306.12327/assets/figures/cosmic-graph.png" width="500" height="500"></center>

**Figure**: A graph of dark matter halos, colored by their halo masses, in a 50 Mpc volume connected using a linking length of 5 Mpc.



**References**
- [Wu & Jespersen (2023)](https://ui.adsabs.harvard.edu/abs/2023arXiv230612327W/abstract) ICML ML4Astro workshop paper; see also the [halo-gnns](https://github.com/jwuphysics/halo-gnns) repository
- [Nelson et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ComAC...6....2N/abstract) for the [Illustris TNG300](https://www.tng-project.org/data/) hydrodynamic simulation

**Other Resources**
- [KITP GNN tutorial notebook](https://github.com/DataDrivenGalaxyEvolution/galevo23-tutorials/blob/main/week-3/KITP-CCA-GNN-tutorial.ipynb) and accompanying [recording](https://www.dropbox.com/scl/fo/tgpwe1ljr4i9go4ajnsq6/h?dl=0&preview=2023-02-03++CCA+KITP+-+Tutorial+-+Christian+Kragh+Jespersen+-+Alpha+Beta.mp4&rlkey=mwl1zi14okj5k1pnbflcgnz3l) by Christian Kragh Jespersen
- [Pytorch-Geometric documentation](https://pytorch-geometric.readthedocs.io/en/latest/) and  [tutorials](https://pytorch-geometric.readthedocs.io/en/latest/get_started/colabs.html)
- [Stanford CS224W tutorials](https://medium.com/stanford-cs224w)

## Setting up the notebook

1. (*Optional*) If you want to use a GPU runtime, make sure to set it before getting started. At the top of this page, click on `Runtime` > `Change runtime type` > Under `Hardware accelerate`, select `T4 GPU`. However, note that Colab only allocates a limited amount of GPU time, and that most of this notebook should run in a reasonable amount of time even if you use a CPU instead.
2. Run the **Install Pytorch-Geometric and other necessary packages** cell below in order to install the required Python packages. You can click the triangle "play button" in order to run each cell. If you see a green checkmark to the left of the cell, then it should have successfully installed the packages! (My cell also says 45s below the checkmark, which is the duration of time it took to install the packages.)
3. Run the **Import Python packages** cell in order to be able to use packages like `pytorch`, `pytorch_geometric`, `numpy`, `pandas`, and `matplotlib`.
4. Run the **Download TNG300 simulation data** cell in order to get the subhalo catalogs that have been put together for this tutorial.



In [ ]:
#@title Install Pytorch-Geometric and other necessary packages
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter -qf https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -qf https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -qf https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -qf https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric -q

In [ ]:
# @title Import Python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import scipy
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader, RandomNodeLoader
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import (
    MessagePassing, GCNConv, SAGEConv, EdgeConv,
)
from torch_cluster import radius_graph
from tqdm import tqdm


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# @title Download TNG300 simulation data
!wget -q -O subhalos.parquet https://www.dropbox.com/scl/fi/7vng9fx9q6gamu4abomhn/subhalos-hydro.parquet?rlkey=r340qt3ju430qlzkr1t5c7gmy&dl=0

## Exploratory data analysis

It's always nice to get some intuition for what's in the data set. Let's get started by checking the number of rows and columns, and sampling some of the rows.

In [ ]:
subhalos = pd.read_parquet("subhalos.parquet")
print(subhalos.shape)
subhalos.sample(5, random_state=42)

Since we know that these are all subhalo properties, I'm going to make the column names more concise.

In [ ]:
subhalo_colname_mapping = {
    c: c[8:]
    for c in subhalos.columns
    if (c.startswith("subhalo_") and c != "subhalo_id")
}

subhalos.rename(subhalo_colname_mapping, axis=1, inplace=True)

subhalos.sample(5, random_state=42)

In [ ]:
# another way to look over the data
subhalos.describe()

In [ ]:
# some of these positions are negative -- let's make sure they're above 0
subhalos[["x", "y", "z"]] -= subhalos[["x", "y", "z"]].min(0)

Let's get a sense of how the data are distributed from a scientific perspective. But we have over half a million halos! Let's get a smaller sample just for visualization.

In [ ]:
# get a smaller subset of rows
df = subhalos[(subhalos.x < 50) & (subhalos.y < 50) & (subhalos.z < 50)].copy()
len(df)

In [ ]:
# @title Let's plot the **Stellar mass-Halo mass relation**:
plt.figure(figsize=(3, 3), dpi=150)
plt.scatter(
    x=df["loghalomass"],
    y=df["logstellarmass"],
    s=5,
    c=np.where(df["is_central"], "C3", "C0"),
    edgecolors="none",
)

plt.text(0.95, 0.2, "Central", color="C3", transform=plt.gca().transAxes, ha="right", fontsize=16)
plt.text(0.95, 0.1, "Satellite", color="C0", transform=plt.gca().transAxes, ha="right", fontsize=16)

plt.xlabel(r"$\log(M_{\rm halo}/M_\odot)$")
plt.ylabel(r"$\log(M_\bigstar/M_\odot)$")
plt.grid(alpha=0.15)

In [ ]:
# @title Let's also take a look at the **3d distribution of galaxies/subhalos**:

fig = plt.figure(figsize=(7, 8), dpi=150)
ax = fig.add_subplot(projection="3d")

fontsize = 12
pos = df[["x", "y", "z"]] - 25

# plot nodes
sc = ax.scatter(
    pos.x,
    pos.y,
    pos.z,
    s=(df.loghalomass-9)**3,
    vmin=9,
    vmax=12,
    alpha=0.8,
    edgecolor='k',
    c=df.logstellarmass,
    cmap="plasma",
    linewidths=0.1
)
fig.subplots_adjust(left=0.1, right=0.9)

cb = fig.colorbar(sc, aspect=50, pad=-0.02, location='top')
cb.set_label(r"log($M_\bigstar/M_\odot$)", fontsize=12)

ax.set_xlim(-25, 25)
ax.set_ylim(-25, 25)
ax.set_zlim(-25, 25)
ax.set_xlabel("X [Mpc]", fontsize=12)
ax.set_ylabel("Y [Mpc]", fontsize=12)
ax.set_zlabel("Z [Mpc]", fontsize=12)
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.xaxis._axinfo["grid"]['color'] =  (0.5,0.5,0.5,0.2)
ax.yaxis._axinfo["grid"]['color'] =  (0.5,0.5,0.5,0.2)
ax.zaxis._axinfo["grid"]['color'] =  (0.5,0.5,0.5,0.2)

## Mathematical graphs

This last plot shows the positions of subhalos in the TNG300 catalog. These data sets are sometimes called "point clouds" in computer vision, and they will serve as the launching point for our foray into graphs and graph neural networks.

Graphs can be represented in many ways, but here we will focus on implementing them in PyG ([`torch_geometric.data.Data`](https://pytorch-geometric.readthedocs.io/en/stable/generated/torch_geometric.data.Data.html#torch_geometric.data.Data)). This `Data` class is a flexible way to store graphs and general features (aka *attributes*). Let's look at the following graph:

In [ ]:
x = torch.tensor([[2.5], [4.5], [1.7], [1.0]], dtype=torch.float)

edge_index = torch.tensor([[0, 1, 1, 2, 1, 3],
                           [1, 0, 2, 1, 3, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index)

print(data)

First, let's look at how the graph is connected via the `data.edge_index` attribute. We'll check out the other attribute, `data.x` (which holds the node features) in a moment.

The `edge_index` is a *sparse* representation of a graph's adjacency matrix, $\mathbf{A}$. The connectivity of the above graph can be described by its adjacency matrix:

$$ \mathbf{A} =
\begin{pmatrix}
0 & 1 & 0 & 0 \\
1 & 0 & 1 & 1 \\
0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0
\end{pmatrix}$$

or equivalently, the `edge_index` representation that we wrote above:

```python
edge_index = torch.tensor([[0, 1, 1, 2, 1, 3],
                           [1, 0, 2, 1, 3, 1]], dtype=torch.long)
```

We can make note of several properties already:
1. The elements of the adjacency matrix $\mathbf{A}_{ij}$ between nodes $i$ and $j$ are 1 if an edge exists, and 0 if an edge doesn't exist between the two nodes.
2. The number of non-zero elements in $\mathbf{A}$ is equal to the length of a row in `edge_index`.
3. The above graph is an *undirected* (aka *bidirected*) graph. As a result, its adjacency matrix $\mathbf{A}$ is symmetric.
4. If the graph has $\mathcal{V}$ nodes and $\mathcal{E}$ *directed* edges, then the graph is sparse when $\mathcal{E} \ll \mathcal{V}^2$. In such cases, the `edge_index` representation (requiring $2 \times \mathcal{E}$ elements) is more efficient than writing out the entire adjacency matrix.
5. None of the nodes connect to themselves, and therefore the diagonal of $\mathbf{A}$ is all zeros. If the graph contains *self-loops*, then those diagonal elements would contain ones.
6. Everything is 0-indexed here because we're using Python. Note that in mathematics we conventionally see 1-indexing.

We can use another package, `networkx`, to make basic visualizations of these graphs. In this case, we are drawing the $\mathcal{V} = 4$ nodes and the $\mathcal{E} = 6$ directed edges. We are also scaling each node's size by its (single) feature stored in `data.x`.

In [ ]:
nxGraph = to_networkx(data)
nx.draw(nxGraph, node_size=(10*data.x)**2, node_color="#ff6361")

### **Exercise 1**
**What is the index of the node that connects to the other three nodes?**


<details>
<summary><b>Answer</b></summary>
  
The node indexed to `1` is connected to the other nodes. You can verify corresponding row and column in the adjacency matrix $\mathbf{A}_{i1}$ and $\mathbf{A}_{1j}^T$ are $(1, 0, 1, 1)$, signifying that the node has undirected edges connecting to the other three nodes.

One way to visualize this is to run the following code
```python
nxGraph = to_networkx(data)
nx.draw(nxGraph, node_size=(10*data.x)**2, node_color="#ff6361", with_labels=True)
```
  
We can also verify that the node feature at the index `1` has the largest value, in accordance with the graph plotted above. By running
```python
for V_i, x_i in enumerate(data.x):
    print(f"Node {V_i}: x = {x_i.item():g}")
```
or by looking back to the constructor for `data`, we can see that the node feature at `data.x[1]` has the largest value.

</details>

### **Exercise 2**

**Create another `torch_geometric.data.Data` object with the same nodes and edge features as above, but also with self-loops.**

<details>
<summary><b>Answer</b></summary>
  
We can construct a new versions of `edge_index` with the self-loops:
```python
edge_index = torch.tensor([[0, 1, 1, 2, 1, 3, 0, 1, 2, 3],
                           [1, 0, 2, 1, 3, 1, 0, 1, 2, 3]], dtype=torch.long)
```
Since the node features remain the same, we don't need to update that. However, we do need to remake the PyG `Data` object, and we can optionally plot it again:
```python
data = Data(x=x, edge_index=edge_index)

nxGraph = to_networkx(data)
nx.draw(nxGraph, node_size=(10*data.x)**2)
```

Another solution is to use [`torch_geometric.utils.add_self_loops`](https://pytorch-geometric.readthedocs.io/en/latest/modules/utils.html#torch_geometric.utils.add_self_loops), which takes in the `edge_index` and an optional `edge_weight`, and returns the output `edge_index` and `edge_weight`.

If we ran it here, we'd find:
```python
from torch_geomegtric.utils import add_self_loops
edge_index, _ = add_self_loops(edge_index)
print(edge_index)

> (tensor([[0, 1, 1, 2, 1, 3, 0, 1, 2, 3],
>         [1, 0, 2, 1, 3, 1, 0, 1, 2, 3]]),
> None)
```

</details>

### **Exercise 3**

**Fill in the `data.edge_index` attribute to connect all nodes (minus self-loops) the graph shown below.**

Note that a graph without self-loops is called a *simple graph*, and a fully connected simple undirected graph is called a *complete graph*.

Hint: You may recall that a complete graph with $\mathcal{V}$ nodes will have $\mathcal{V}(\mathcal{V}-1)/2$ undirected edges, but using PyG we need to specify both directions for the edges!


In [ ]:
# add indices below
edge_index = torch.tensor([[], []], dtype=torch.long)

# just stick random entries below
x_features = torch.randn((4, 1))

data = Data(edge_index=edge_index, x=x_features)

nxGraph = to_networkx(data)
nx.draw(nxGraph)

<details>
<summary><b>Answer</b></summary>
  
One solution is to write out all of the indices, e.g.,

```python
edge_index = torch.tensor([[1, 2, 3, 2, 3, 3, 0, 1, 2, 0, 1, 0],
                           [0, 1, 2, 0, 1, 0, 1, 2, 3, 2, 3, 3]], dtype=torch.long)
```

Another solution is to write out half of these edges, and then use the helper function [`torch.utils.to_undirected`](https://pytorch-geometric.readthedocs.io/en/latest/modules/utils.html#torch_geometric.utils.to_undirected) like such:
```python
from torch_geometric.utils import to_undirected

edge_index = torch.tensor([[1, 2, 3, 2, 3, 3],
                           [0, 1, 2, 0, 1, 0]], dtype=torch.long)
edge_index = to_undirected(edge_index)
```

Finally we can construct our `Data` object, and plot it:
```python
x_features = torch.randn((4,1))

data = Data(edge_index=edge_index, x=x_features)

nxGraph = to_networkx(data)
nx.draw(nxGraph)
```
</details>

### Some final technical notes
* We are writing out the edge indices by specifying all of the $i$ indices, and then all of the $j$ indices. It is also possible to write out a tensor of $(i, j)$ indices, and then transpose that tensor. However, we must ensure that this representation is *contiguous* in memory, e.g.:

    ```python
  x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

  edge_index = torch.tensor([[0, 1],
                             [1, 0],
                             [1, 2],
                             [2, 1]], dtype=torch.long)

  data = Data(x=x, edge_index=edge_index.t().contiguous())
    ```
* For indices and other integer-typed attributes, make sure you specify the data type as `dtype=torch.long`.
* This tutorial only focuses on homogeneous graphs, i.e., all of the nodes correspond to galaxies/subhalos, and all edges encode relationships between them. However, it's possible to build graphs with heterogeneous nodes or edges. For example, the Atacama Large Millimeter/submillimeter Array (ALMA) comprises both 7-m and 12-m antennas (see below); they could be represented on [*heterogeneous graphs*](https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html?highlight=heterogeneous).
<center><img src="https://www.almaobservatory.org/wp-content/uploads/2023/05/file-1-copy-2048x988.jpeg" width="600"></center>


## Building a cosmic graph using PyG

In [ ]:
# select a smaller subset of subhalos that are better resolved by the simulation
cut = subhalos["loghalomass"] > 11

df = subhalos[cut].copy()

In [ ]:
# get input node features, `x`, and targets, `y`
x = torch.tensor(df[["loghalomass", "logvmax"]].values, dtype=torch.float)
y = torch.tensor(df[["logstellarmass"]].values, dtype=torch.float)

# also keep track of positions
pos = torch.tensor(df[["x", "y", "z"]].values, dtype=torch.float)

In [ ]:
D_link = 5

# get all edges
kd_tree = scipy.spatial.KDTree(pos, leafsize=25, boxsize=205.0001/0.6774)
edge_index = kd_tree.query_pairs(r=D_link, output_type="ndarray")

# use tensor, add reverse pairs
edge_index = torch.Tensor(edge_index).t().contiguous().type(torch.long)
edge_index = to_undirected(edge_index)

In [ ]:
# compute distances and velocity differences between pairwise node combinations
row, col = edge_index
dist = torch.tensor(np.linalg.norm(pos[row] - pos[col], axis=1)).reshape(-1, 1)

vel = torch.tensor(df[["vx", "vy", "vz"]].values, dtype=torch.float)
veldiff = torch.tensor(np.linalg.norm(vel[row] - vel[col], axis=1)).reshape(-1, 1)

edge_attr = torch.cat([dist, veldiff], dim=-1).type(torch.float)

In [ ]:
data = Data(
    x=x,
    y=y,
    pos=pos,
    edge_index=edge_index,
    edge_attr=edge_attr,
)

In [ ]:
data

## Setting up the problem: Predict $M_\star$ from $M_{\rm halo}$ with GNNs

In [ ]:
# a smaller subset of the full data
train_mask = (df["z"] <= 200) & (df["x"] < 50) & (df["y"] < 50)
valid_mask = (df["z"] > 210) & (df["x"] < 50) & (df["y"] < 50)

# note -- this is the equivalent of `np.argwhere()`
train_indices = torch.tensor(train_mask.values).nonzero(as_tuple=True)[0]
valid_indices = torch.tensor(valid_mask.values).nonzero(as_tuple=True)[0]

assert (set(train_indices) & set(valid_indices)) == set()

## Train a baseline model

In this case, we will train a [`scikit-learn` Random Forest (RF) regression model](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor). Refer back to Viviana's lectures for details on decision tree-based algorithms!

In [ ]:
# train-test split
df_train = df[train_mask].copy()
df_valid = df[valid_mask].copy()

X_train = df_train[["loghalomass", "logvmax"]].values
y_train = df_train[["logstellarmass"]].values
X_valid = df_valid[["loghalomass", "logvmax"]].values
y_valid = df_valid[["logstellarmass"]].values

In [ ]:
# random forest <3
rf = RandomForestRegressor(n_estimators=25, random_state=42)
rf.fit(X_train, y_train.ravel())
p_valid = rf.predict(X_valid)

In [ ]:
plt.figure(figsize=(5, 4), dpi=150)
plt.scatter(y_valid, p_valid, s=5, edgecolors="none", c=df_valid.loghalomass.values, vmin=11, vmax=14)
plt.colorbar(label=r"$\log(M_{\rm halo}/M_\odot)$")
plt.grid(alpha=0.15)
plt.xlim(9, 12)
plt.ylim(9, 12)
plt.xlabel(r"True $\log(M_\bigstar/M_\odot)$", fontsize=12)
plt.ylabel(r"Predicted $\log(M_\bigstar/M_\odot)$", fontsize=12)

plt.title(f"RMSE(RF) = {((p_valid.ravel() - y_valid.ravel())**2).mean()**0.5:.4f} dex");

## Creating a simple graph neural network

This network uses SAGEConv (or GraphSAGE), which uses node-only features and the graph connectivity. In other words, the subhalos know about their own and neighboring subhalos' masses, but nothing about positional or velocity differences between it and its neighbors. See the paper by [Hamilton et al. (2017)](https://arxiv.org/abs/1706.02216) for more details.

In [ ]:
class SAGEGraphConvNet(torch.nn.Module):
    """A simple GNN built using SAGEConv layers.
    """
    def __init__(self, n_in=2, n_hidden=128, n_out=1):
        super(SAGEGraphConvNet, self).__init__()
        self.conv1 = SAGEConv(n_in, n_hidden)
        self.conv2 = SAGEConv(n_hidden, n_hidden)
        self.fc = nn.Linear(n_hidden, n_out, bias=True)
        self.mlp = nn.Sequential(
            nn.Linear(n_in, n_hidden, bias=True),
            nn.ReLU(),
            nn.LayerNorm(n_hidden),
            nn.Linear(n_hidden, n_out, bias=True)
        )

    def forward(self, data):
        x0, edge_index = data.x, data.edge_index

        x1 = self.conv1(x0, edge_index)
        x2 = self.conv2(F.relu(x1), edge_index)
        convs_out = self.fc(F.relu(x2))
        mlp_out = self.mlp(x0)
        return convs_out + mlp_out

In [ ]:
model = SAGEGraphConvNet().to(device)

### **Exercise 4**

**What are the dimensions of the inputs and outputs for each layer in the initialized model (`conv1`, `conv2`, `fc`, and `mlp`)?**



<details>
<summary><b>Answer</b></summary>

All of the layers are two-dimensions, with the first dimension equal to the batch size, which I'll call `bs`.

For `conv1`: The inputs are of shape `(bs, 2)` since there are two node features. The outputs are of shape `(bs, 128)`, since `n_hidden = 128` when we initialized the model.

For `conv2`: The intputs are now shape `(bs, 128)` since it takes the outputs of the previous layer (after applying a rectified linear unit, or ReLU). The output is again `(bs, 128)`.

For `fc`: The inputs have shape `(bs, 128)` since it receives the previous layer's output. The output has shape `(bs, 1)`.

For `mlp`: The inputs have shape `(bs, 2)`, as it ingests the node-level features as inputs. There is a hidden layer, non-linear activation, and normalization layer, and finally the output is of shape `(bs, 1)`.

</details>

### **Exercise 5**

**Modify the SAGEGraphConvNet in the following way:**


1. **Remove the `fc` layer**.
2. **Adjust the `mlp` layer such that it ingests both the node input features, as well as the activated outputs of `conv2`**.

*Hint: for part two, you'll need to concatenate the features using `torch.cat()` along the last dimension.*



<details>
<summary><b>Answer</b></summary>

One solution is like such:

```python
class SAGEGraphConvNet(torch.nn.Module):
    """A simple GNN built using SAGEConv layers.
    """
    def __init__(self, n_in=2, n_hidden=16, n_out=1):
        super(SAGEGraphConvNet, self).__init__()
        self.conv1 = SAGEConv(n_in, n_hidden)
        self.conv2 = SAGEConv(n_hidden, n_hidden)
        self.mlp = nn.Sequential(
            nn.Linear(n_in + n_hidden, n_hidden, bias=True),
            nn.ReLU(),
            nn.LayerNorm(n_hidden),
            nn.Linear(n_hidden, n_out, bias=True)
        )

    def forward(self, data):
        x0, edge_index = data.x, data.edge_index

        x1 = self.conv1(x0, edge_index)
        x2 = self.conv2(F.relu(x1), edge_index)
        return self.mlp(torch.cat([F.relu(x2), x0], dim=-1))
```
</details>

In [ ]:
def train(dataloader, model, optimizer, device="cuda"):

    model.train()

    loss_total = 0
    for data in dataloader:
        data.to(device)

        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.mse_loss(y_pred, data.y)

        loss.backward()
        optimizer.step()
        loss_total += loss.item()

    return loss_total / len(dataloader)

def validate(dataloader, model, device="cuda"):
    model.eval()

    uncertainties = []
    loss_total = 0

    y_preds = []
    y_trues = []

    for data in dataloader:
        with torch.no_grad():
            data.to(device)
            y_pred = model(data)
            loss = F.mse_loss(y_pred, data.y)

            loss_total += loss.item()
            y_preds += list(y_pred.detach().cpu().numpy())
            y_trues += list(data.y.detach().cpu().numpy())

    y_preds = np.concatenate(y_preds)
    y_trues = np.array(y_trues)

    return (
        loss_total / len(dataloader),
        y_preds,
        y_trues,
    )

In [ ]:
bs = 256
lr = 1e-2

n_epochs = 100

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=lr,
)

train_loader = RandomNodeLoader(
    data.subgraph(train_indices),
    num_parts=(len(train_indices) // bs),
    shuffle=True
)

valid_loader = RandomNodeLoader(
    data.subgraph(valid_indices),
    num_parts=(len(valid_indices) // bs),
    shuffle=False
)


In [ ]:
train_losses = []
valid_losses = []

print(f"Epoch    Train loss   Valid Loss      RSME ")
for epoch in range(n_epochs):

    train_loss = train(train_loader, model, optimizer, device=device)
    valid_loss, p, y  = validate(valid_loader, model, device=device)

    print(f" {epoch + 1: >4d}    {train_loss: >9.5f}    {valid_loss: >9.5f}    {np.sqrt(np.mean((p - y.flatten())**2)): >10.6f}")
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

In [ ]:
valid_loss, p, y  = validate(valid_loader, model, device=device)

plt.figure(figsize=(5, 4), dpi=150)
plt.scatter(y, p, s=5, edgecolors="none", c=df.iloc[valid_indices].loghalomass.values, vmin=11, vmax=14)
plt.colorbar(label=r"$\log(M_{\rm halo}/M_\odot)$")
plt.grid(alpha=0.15)
plt.xlim(9, 12)
plt.ylim(9, 12)
plt.xlabel(r"True $\log(M_\bigstar/M_\odot)$", fontsize=12)
plt.ylabel(r"GNN Predicted $\log(M_\bigstar/M_\odot)$", fontsize=12)

plt.title(f"RMSE(Simple GNN) = {((p.ravel() - y.ravel())**2).mean()**0.5:.4f} dex");

There are many things to vary here in order to help build intuition, for example:
- `SAGEConv` allows you to change the aggregation function from the default of `mean` pooling to anything else.
- What happenes when you change the number of hidden layers in the network?
- What happens when you increase or decrease the batch size?
- How do these hyperparameters (qualitatively) covary with each other?

## Bonus! A far more complicated GNN with edge features and node-edge interactions

Check out the GNN implemented below, which is based on the one from the [Wu & Jespersen (2023)](https://arxiv.org/abs/2306.12327) paper. Can you sketch out the architecture based on the code?

In [ ]:
class EdgeInteractionLayer(MessagePassing):
    """Interaction network layer with node + edge layers.
    """
    def __init__(self, n_in, n_hidden, n_latent, aggr='sum'):
        super(EdgeInteractionLayer, self).__init__(aggr)

        self.mlp = nn.Sequential(
            nn.Linear(n_in, n_hidden, bias=True),
            nn.LayerNorm(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden, bias=True),
            nn.LayerNorm(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_latent, bias=True),
        )

        self.messages = 0.
        self.input = 0.

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr):

        self.input = torch.cat([x_i, x_j, edge_attr], dim=-1)
        self.messages = self.mlp(self.input)

        return self.messages

class EdgeInteractionGNN(nn.Module):
    """Graph net over nodes and edges with multiple unshared layers, and sequential layers with residual connections.
    Self-loops also get their own MLP (i.e. galaxy-halo connection).
    """
    def __init__(self, n_layers, D_link, node_features=2, edge_features=2, hidden_channels=16, aggr="sum", latent_channels=16, n_out=1, n_unshared_layers=8, loop=True, use_global_pooling=True):
        super(EdgeInteractionGNN, self).__init__()

        self.n_in = 2 * node_features + edge_features
        self.n_out = n_out
        self.use_global_pooling = use_global_pooling

        layers = [
            nn.ModuleList([
                EdgeInteractionLayer(self.n_in, hidden_channels, latent_channels, aggr=aggr)
                for _ in range(n_unshared_layers)
            ])
        ]
        for _ in range(n_layers-1):
            layers += [
                nn.ModuleList([
                    EdgeInteractionLayer(2 * latent_channels * n_unshared_layers + node_features, hidden_channels, latent_channels, aggr=aggr)
                    for _ in range(n_unshared_layers)
                ])
            ]

        self.layers = nn.ModuleList(layers)

        n_pool = (len(aggr) if isinstance(aggr, list) else 1)
        self.fc = nn.Sequential(
            nn.Linear((n_unshared_layers * n_pool) * latent_channels, latent_channels, bias=True),
            nn.LayerNorm(latent_channels),
            nn.ReLU(),
            nn.Linear(latent_channels, latent_channels, bias=True),
            nn.LayerNorm(latent_channels),
            nn.ReLU(),
            nn.Linear(latent_channels, latent_channels, bias=True)
        )

        self.galaxy_halo_mlp = nn.Sequential(
            nn.Linear(latent_channels + node_features, latent_channels, bias=True),
            nn.LayerNorm(latent_channels),
            nn.ReLU(),
            nn.Linear(latent_channels, latent_channels, bias=True),
            nn.LayerNorm(latent_channels),
            nn.ReLU(),
            nn.Linear(latent_channels, n_out, bias=True)
        )

        self.D_link = D_link
        self.h = 0.

    def forward(self, data):
        edge_index = data.edge_index
        edge_attr = data.edge_attr

        # update hidden state on edge (h, or sometimes e_ij in the text)
        h = torch.cat([unshared_layer(data.x, edge_index=edge_index, edge_attr=edge_attr) for unshared_layer in self.layers[0]], axis=1)
        self.h = h
        h = h.relu()

        for layer in self.layers[1:]:
            # if more than 1 layer, then also use a skip connection
            h = self.h + torch.cat([unshared_layer(h, edge_index=edge_index, edge_attr=edge_attr) for unshared_layer in layer], axis=1)

            self.h = h
            h = h.relu()

        x = torch.concat([self.fc(h), data.x], axis=1) # latent channels + data.x

        return (self.galaxy_halo_mlp(x))

In [ ]:
node_features = data.x.shape[1]
edge_features = data.edge_attr.shape[1]


bs = 256
lr = 3e-2

n_epochs = 100

n_layers = 1
n_hidden_channels = 32
n_latent_channels = 32
n_unshared_layers = 8

model = EdgeInteractionGNN(
    node_features=node_features,
    edge_features=edge_features,
    n_layers=n_layers,
    D_link=D_link,
    hidden_channels=n_hidden_channels,
    latent_channels=n_latent_channels,
    loop=True,
    n_unshared_layers=n_unshared_layers,
    n_out=1,
    aggr="sum"
)
model.to(device);

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=lr,
)

train_loader = RandomNodeLoader(
    data.subgraph(train_indices),
    num_parts=(len(train_indices) // bs),
    shuffle=True
)

valid_loader = RandomNodeLoader(
    data.subgraph(valid_indices),
    num_parts=(len(valid_indices) // bs),
    shuffle=False
)

In [ ]:
train_losses = []
valid_losses = []

print(f"Epoch    Train loss   Valid Loss      RSME ")
for epoch in range(n_epochs):

    # anneal learning rate
    if epoch == int(n_epochs * 0.5):
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr / 10,
        )
    elif epoch == int(n_epochs * 0.9):
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr / 100,
        )

    train_loss = train(train_loader, model, optimizer, device=device)
    valid_loss, p, y  = validate(valid_loader, model, device=device)

    print(f" {epoch + 1: >4d}    {train_loss: >9.5f}    {valid_loss: >9.5f}    {np.sqrt(np.mean((p - y.flatten())**2)): >10.6f}")
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

In [ ]:
plt.plot(np.sqrt(train_losses), label="Train")
plt.plot(np.sqrt(valid_losses), label="Valid")
plt.xlabel("Epoch")
plt.ylabel("RMSE [dex]")
plt.legend(fontsize=14)
plt.grid(alpha=0.15)
plt.ylim(0.15, 0.25)
plt.title("Loss curves")

In [ ]:
valid_loss, p, y  = validate(valid_loader, model, device=device)

plt.figure(figsize=(5, 4), dpi=150)
plt.scatter(y, p, s=5, edgecolors="none", c=df.iloc[valid_indices].loghalomass.values, vmin=11, vmax=14)
plt.colorbar(label=r"$\log(M_{\rm halo}/M_\odot)$")
plt.grid(alpha=0.15)
plt.xlim(9, 12)
plt.ylim(9, 12)
plt.xlabel(r"True $\log(M_\bigstar/M_\odot)$", fontsize=12)
plt.ylabel(r"GNN Predicted $\log(M_\bigstar/M_\odot)$", fontsize=12)

plt.title(f"RMSE(Advanced GNN) = {((p.ravel() - y.ravel())**2).mean()**0.5:.4f} dex");

### **Exercse 6**
**Add random noise to features during training. Does it help?**

Rationale: Injecting a small amount of random noise to the node and edge features can help the model learn a more robust representation. This method is sometimes called *noise augmentation* or *noise regularization*.

<details>
<summary><b>Answer</b></summary>

Add the following lines to the `train()` train after loading each batch of data:

```python
if augment:
    data_node_features_scatter = 1e-3 * torch.randn_like(data.x) * torch.std(data.x, dim=0)
    data_edge_features_scatter = 1e-3 * torch.randn_like(data.edge_attr) * torch.std(data.edge_attr, dim=0)

    data.x += data_node_features_scatter
    data.edge_attr += data_edge_features_scatter
```

This also introduces another hyperparameter, that is, the scale of noise added during training time. Here I chose `1e-3`, but you may find that other values are better.
</details>

### **Exercise X**

**Design a heterogeneous dataset and graph network that learns different relationships for *central* and *satellite* subhalos.**

Sorry, no solution here. This would be an interesting research problem though!